<a href="https://colab.research.google.com/github/aviax1/bark_detector/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**dependencies**

In [ ]:
!pip install wandb

In [53]:
import torch,wandb,os,warnings,csv
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

**get data**

In [ ]:
!git clone https://github.com/aviax1/bark_detector/

In [56]:
pd.plotting.register_matplotlib_converters()
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

def read_fv():
    size=8732
    X_ = []
    Y = []
    i = 0
    with open('bark_detector/feature_vector.csv', newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            if(i > 0):
                rw=row[0].split(",")
                fv=rw[2:]
                temp = np.array(fv)
                fv = temp.transpose()
                X_.append(fv)
                Y.append(int(rw[1]))
            i+=1
    X = np.empty([size, 128])
    for i in range(size):
        X[i] = (X_[i])
    #Y = to_categorical(Y)
    xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state = 1)
    xtrain = xtrain.reshape(6549, 128, 1)
    xtest = xtest.reshape(2183,128, 1)
    return xtrain, xtest, np.array(ytrain), np.array(ytest)

**load data**

In [65]:
(xtrain2,ytrain2), (xtest2,ytest2) = mnist.load_data() #old data

xtrain, xtest, ytrain, ytest=read_fv() # new data
num_epochs=1000        #
batch_size = 64        #
image_size=128         #
hidden_size=32         #
lv_size = 6            # Latent Variable 
learning_rate=1e-8     #
cret = nn.MSELoss()    # criterion
warnings.filterwarnings('ignore')

**build model**

In [58]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size), 
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
             nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
             nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**model setting**

In [66]:
model = autoencoder()
tmodel=autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-7)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])

**classsifcation by train models**

In [ ]:
def model_name(digit):
  return './ae_'+str(digit)+'.pth'

def get_prediction(data=xtest):
  nn=len(data)
  dataloader = DataLoader(DigitDataSet(data), batch_size=nn,shuffle=0 , num_workers=4)
  diff = np.zeros( (nn,10),dtype=np.float32 )
  for i in range(10):
    for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      imgs=imgs.float()
      tmodel.load_state_dict(torch.load(model_name(i)))
      tmodel.eval()
      output_imgs = tmodel(imgs)
      for i2 in range(len( output_imgs[:,0])):
        im_pred=output_imgs.detach().numpy()[i2,:]
        im_org=imgs.numpy()[i2,:]
        difmat=np.abs(im_pred.reshape(128,1)-im_org.reshape(128,1))
        diff[i2,i]=np.sum( np.sum( difmat ))
  return np.argmin(diff, axis=1)


def testmodel(): 
  nn=len(ytest)
  min_index =get_prediction()
  seccess =  min_index == ytest
  counts, bins = np.histogram(ytest[ min_index != ytest ])
  plt.hist(bins[:-1], bins, weights=counts)
  plt.title("error by digit")
  plt.show()
  accurcy =int(10000*np.sum(seccess))/(nn*100)
  error_rate = int(10000*np.sum(min_index != ytest))/(nn*100)
  print(str(accurcy) + "% accuracy or "+str(error_rate)+"% error rate")
  return counts, bins ,len(ytest[min_index != ytest]) , len(ytest)

testmodel()

**train method**

In [ ]:
def save_model(digit,model):
  mn=model_name(digit)
  torch.save(model.state_dict(),mn )
  wandb.save(mn)
  print("save model "+ mn)

def load_model_ifexist(digit,model):
  mn=model_name(digit)
  if os.path.isfile(mn):
    model.load_state_dict(torch.load(mn))
    model.eval()
  return model

def train_by_digit(by_digit,model,ne=num_epochs,opt=optimizer):
  model=load_model_ifexist( by_digit,model)
  wandb.init()
  print("*****\nstart traning Model for digit " +str(by_digit) +"\n")
  dataloader = DataLoader(DigitDataSet(xtrain[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=6)
  minloss=100000000
  for epoch in range(ne):
    run=  epoch%25==0
    run2= epoch%125==0 and epoch >0
    for data in dataloader:
      
      imgs = Variable(data.view(data.size(0), -1))
      imgs=imgs.float()
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      
      if minloss!=100000000:
        opt.zero_grad()
        loss.backward()
        opt.step()
      if run:
        run=0
        im=data[0,0,:,:].reshape(128,1)
        pred=model(imgs).detach().numpy()[0,:].reshape(128,1)
        wandb.log({"img": [wandb.Image(pred, caption="preidciton"),wandb.Image(im, caption="original")]})
      if run2:
        run2=0
        save_model(by_digit,model)
        testmodel()
    newlost = float(loss.data ) 
    if newlost < minloss:
        if minloss!=100000000:
          save_model(by_digit,model)
        minloss=newlost
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, ne, loss.data))
    wandb.log({"loss": loss.data})
  
  print("\nfinish traning Model Number " +str(by_digit) +"\n*****\n")

train_by_digit(0,model,50)

**train new model**

In [ ]:
for by_digit in range(10):
  train_by_digit(by_digit,model,120)
_,_,_,_=testmodel()

**or used our train model**

In [ ]:
!git clone https://github.com/aviax1/AE1/
!unzip ./AE1/models.zip -d ./
!rm -rf ./AE1

**finaly test model**

In [ ]:
_,_,_,_=testmodel()

**retrain the model**

In [ ]:
traget_error=0.005
error = 1
while  error > traget_error:
  counts,b,fail,total=testmodel()
  error = float(fail/total)
  if error > traget_error:
    train_by_digit(np.argmax(counts),model,30, torch.optim.Adam(model.parameters(), lr=1e-3) )

**kaggle**

In [ ]:
!git clone https://github.com/aviax1/AE1/
!unzip ./AE1/kaggle.zip -d ./
!rm -rf ./AE1

In [ ]:
inputs_test=pd.read_csv('./test.csv')
inputs_test=np.array(inputs_test,dtype=np.float32)
inputs_test=inputs_test.reshape(inputs_test.shape[0],28,28)/255
y=get_prediction(inputs_test)
imageid=1
with open('submission.csv', 'w', newline='') as csvfile:
  spamwriter = csv.writer(csvfile, delimiter=' ',    quotechar='|', quoting=csv.QUOTE_MINIMAL)
  spamwriter.writerow(['ImageId,Label'])
  for yi in y:
    spamwriter.writerow([str(imageid) +','+str( yi)])
    imageid+=1
#99.714% accuracy